# Part I. ETL Pipeline for Pre-Processing the Files

## PLEASE RUN THE FOLLOWING CODE FOR PRE-PROCESSING THE FILES

#### Import Python packages 

In [1]:
# Import Python packages 
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv

#### Creating list of filepaths to process original event csv data files

In [2]:
# checking your current working directory
print(os.getcwd())

# Get your current folder and subfolder event data
filepath = os.getcwd() + '/event_data'

# Create a for loop to create a list of files and collect each filepath
for root, dirs, files in os.walk(filepath):
    
# join the file path and roots with the subdirectories using glob
    file_path_list = glob.glob(os.path.join(root,'*'))
    #print(file_path_list)

/home/workspace


#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [3]:
# initiating an empty list of rows that will be generated from each file
full_data_rows_list = [] 
    
# for every filepath in the file path list 
for f in file_path_list:

# reading csv file 
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        # creating a csv reader object 
        csvreader = csv.reader(csvfile) 
        next(csvreader)
        
 # extracting each data row one by one and append it        
        for line in csvreader:
            #print(line)
            full_data_rows_list.append(line) 
            
# uncomment the code below if you would like to get total number of rows 
#print(len(full_data_rows_list))
# uncomment the code below if you would like to check to see what the list of event data rows will look like
#print(full_data_rows_list)

# creating a smaller event data csv file called event_datafile_full csv that will be used to insert data into the \
# Apache Cassandra tables
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))


In [4]:
# check the number of rows in your csv file
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

6821


# Part II. Complete the Apache Cassandra coding portion of your project. 

## Now you are ready to work with the CSV file titled <font color=red>event_datafile_new.csv</font>, located within the Workspace directory.  The event_datafile_new.csv contains the following columns: 
- artist 
- firstName of user
- gender of user
- item number in session
- last name of user
- length of the song
- level (paid or free song)
- location of the user
- sessionId
- song title
- userId

The image below is a screenshot of what the denormalized data should appear like in the <font color=red>**event_datafile_new.csv**</font> after the code above is run:<br>

<img src="images/image_event_datafile_new.jpg">

## Begin writing your Apache Cassandra code in the cells below

#### Creating a Cluster

In [5]:
# This should make a connection to a Cassandra instance your local machine 
# (127.0.0.1)

from cassandra.cluster import Cluster
cluster = Cluster()

# To establish connection and begin executing queries, need a session
session = cluster.connect()

#### Create Keyspace

In [6]:
try:
    session.execute("""
    CREATE KEYSPACE IF NOT EXISTS udacity 
    WITH REPLICATION = 
    { 'class' : 'SimpleStrategy', 'replication_factor' : 1 }"""
)

except Exception as e:
    print(e)

#### Set Keyspace

In [7]:
try:
    session.set_keyspace('udacity')
except Exception as e:
    print(e)

### Now we need to create tables to run the following queries. Remember, with Apache Cassandra you model the database tables on the queries you want to run.

## Create queries to ask the following three questions of the data

### 1. Give me the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4


### 2. Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182
    

### 3. Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'




In [53]:
query = "CREATE TABLE IF NOT EXISTS music_library"
query = query + "(sessionId int, itemInSession int, artist_name text, song text, firstName text, lastName text, length float, PRIMARY KEY (sessionId, itemInSession))"
try:
    session.execute(query)
except Exception as e:
    print(e)
                   

- <font color=seagreen>For `music_libray` the column `sessionId` was used as primary key so the data is distribuited and ordered by sessionid</font>
- <font color=seagreen >`itemInSession` was used as a clustering column to ensure uniqueness</font>

In [21]:
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:

        query = "INSERT INTO music_library (sessionId, itemInSession, artist_name, song, firstName, lastName, length)"
        query = query + " VALUES (%s, %s, %s, %s, %s, %s, %s)"

        session.execute(query, (int(line[8]), int(line[3]), str(line[0]), str(line[8]), str(line[1]), str(line[4]), float(line[5])))

#### Do a SELECT to verify that the data have been inserted into each table

- <font color=seagreen >For the Select statement the exact columns that needs to be shown is listed with the intention that query be as optimized as possible</font>

- <font color=seagreen >The table `music_library` will need artist, song, firstName, lastName, because that is the information that will be requested. We also need userId, sessionId, itemInSession because that is how the results will be filtered and ordered.</font>


In [22]:
query = "select sessionId, itemInSession, artist_name, song, firstName, lastName, length from music_library WHERE sessionId = 338 and itemInSession = 4"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
for row in rows:
    print (row.sessionid, row.iteminsession, row.artist_name, row.song, row.firstname, row.lastname)

338 4 Faithless 338 Ava Robinson


### COPY AND REPEAT THE ABOVE THREE CELLS FOR EACH OF THE THREE QUESTIONS

###  <font color=#0099ff>Intended Query: </font>

>Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182

In [54]:
query = "CREATE TABLE IF NOT EXISTS artist_song_library"
query = query + "(userid int,sessionid int, iteminsession int,artist_name text, song text, firstName text, lastName text, PRIMARY KEY ( (userId, sessionId), itemInSession))"
try:
    session.execute(query)
except Exception as e:
    print(e)                    

In [55]:
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:

        query = "INSERT INTO artist_song_library ( userid,sessionid, iteminsession,artist_name, song, firstName, lastName)"
        query = query + " VALUES (%s, %s, %s, %s, %s, %s, %s)"
        session.execute(query, ( int(line[10]), int(line[8]),int(line[3]),str(line[0]), str(line[9]), str(line[1]), str(line[4])))

- <font color=seagreen >To display all artists, songs and  user information filtering by user and session, the primary key should by a composite key by the columns `userid` and `sessionid`</font>
- <font color=seagreen >The third key `iteminsession` will be used for ordering the data as requested</font>

In [58]:
query = "select iteminsession , artist_name, song, firstname, lastname from artist_song_library WHERE userid = 10 and sessionid = 182"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
for row in rows:
    print (row.iteminsession,row.artist_name, row.song, row.firstname, row.lastname)

0 Down To The Bone Keep On Keepin' On Sylvie Cruz
1 Three Drives Greece 2000 Sylvie Cruz
2 Sebastien Tellier Kilometer Sylvie Cruz
3 Lonnie Gordon Catch You Baby (Steve Pitron & Max Sanna Radio Edit) Sylvie Cruz


### <font color=#0099ff>Intended Query:</font>
>Query 3: Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'


In [34]:
query = "CREATE TABLE IF NOT EXISTS usernames_by_music"
query = query + "(userid int, song text, firstName text, lastName text, PRIMARY KEY (song,userid))"
try:
    session.execute(query)
except Exception as e:
    print(e)  

In [60]:
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = "INSERT INTO usernames_by_music (song,userid,  firstName, lastName)"
        query = query + " VALUES (%s, %s, %s, %s)"
        session.execute(query, ( str(line[9]),int(line[10]), str(line[1]), str(line[4])))

 - <font color=seagreen>To display every user information filtering by song the primary key should by the column `song`</font>
 - <font color=seagreen>The column `userid` was also used as composed primary key since its already a unique ons its own ensuring that every user the listened to the filtered wll be listed</font>

In [37]:
query = "select firstName, lastName from usernames_by_music WHERE song='All Hands Against His Own'"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
for row in rows:
    print (row.firstname, row.lastname)
                    

Jacqueline Lynch
Tegan Levine
Sara Johnson


### Drop the tables before closing out the sessions

In [33]:
query = "drop table music_library"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
query1 = "drop table artist_song_library"
try:
    rows = session.execute(query1)
except Exception as e:
    print(e)
    
query2 = "drop table usernames_by_music"
try:
    rows = session.execute(query2)
except Exception as e:
    print(e)

Error from server: code=2200 [Invalid query] message="unconfigured table music_library"
Error from server: code=2200 [Invalid query] message="unconfigured table artist_song_library"
Error from server: code=2200 [Invalid query] message="unconfigured table music_usernames"


### Close the session and cluster connection¶

In [62]:
session.shutdown()
cluster.shutdown()